# Mozambique AA for Cholera – Threshold Analysis

This notebook supports the development of Anticipatory Action (AA) for cholera in Mozambique. It focuses on exploring and validating outbreak thresholds as defined in the national cholera preparedness and response plan.

The analysis leverages historical cholera case data and related risk indicators stored securely in Azure Blob Storage. Due to the sensitivity of the data, the data is not publicly available.

**Objectives:**
- Understand the thresholds set by the national plan and assess their applicability.
- Explore alternative data-driven thresholds to inform early action.

> **Note:** Ensure that access credentials for the Blob Storage container are configured before running the data loading section.


In [3]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [53]:
# Data and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import re

# Azure Blob Storage
from azure.storage.blob import ContainerClient, BlobClient
import io

# Display settings
pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:,.0f}".format
sns.set(style="whitegrid")

In [ ]:
DEV_BLOB_SAS = os.getenv("DSCI_AZ_BLOB_DEV_SAS")
DEV_BLOB_NAME = "imb0chd0dev"
DEV_BLOB_URL = f"https://{DEV_BLOB_NAME}.blob.core.windows.net/"
CONTAINER_NAME = "projects"
BLOB_PATH = "ds-aa-moz-cholera/raw/"
BLOB_PATH_WRITE = "ds-aa-moz-cholera/processed/"
container_url = f"{DEV_BLOB_URL}{CONTAINER_NAME}?{DEV_BLOB_SAS}"
container_client = ContainerClient.from_container_url(container_url)

In [49]:
excel_blobs = [
    blob.name
    for blob in container_client.list_blobs(name_starts_with=BLOB_PATH)
    if blob.name.endswith((".xls", ".xlsx"))
]
dataframes = {}
for blob_name in excel_blobs:
    blob_url = f"{DEV_BLOB_URL}{CONTAINER_NAME}/{blob_name}?{DEV_BLOB_SAS}"
    blob_client = BlobClient.from_blob_url(blob_url)
    blob_data = blob_client.download_blob().readall()
    ext = os.path.splitext(blob_name)[1].lower()

    if ext in [".xls", ".xlsx"]:
        xls = pd.ExcelFile(io.BytesIO(blob_data))
        key = os.path.basename(blob_name).replace(ext, "")
        dataframes[key] = xls

## Data Cleaning

In [ ]:
cleaned_data = []

for file_key, xls_file in dataframes.items():
    if "zambezia" in file_key.lower():
        for sheet in xls_file.sheet_names:
            sheet_df = xls_file.parse(sheet, header=None)
            weeks = sheet_df.iloc[1, 1:53].values  # Row 2, columns B to BA

            for row in range(2, 7):  # Rows 3 to 7 (years)
                year = sheet_df.iloc[row, 0]
                values = sheet_df.iloc[row, 1:53].values

                for week, value in zip(weeks, values):
                    # Clean week value
                    if isinstance(week, str):
                        week = week.strip()
                        if week.isdigit():
                            week_clean = int(week)
                        elif week.lower().startswith(("w", "s")):
                            digits = re.sub(r"\D", "", week)
                            week_clean = int(digits) if digits else None
                        else:
                            week_clean = None
                    elif pd.notna(week):
                        week_clean = int(week)
                    else:
                        week_clean = None

                    cleaned_data.append(
                        {
                            "province": "Zambezia",
                            "district": sheet.strip(),
                            "year": int(year),
                            "week": week_clean,
                            "cases": value,
                        }
                    )

    elif "nampula" in file_key.lower():
        for sheet in xls_file.sheet_names:
            sheet_df = xls_file.parse(sheet, header=None)

            # Dynamically find the row with year headers (e.g., B1–H1 or B2–H2)
            year_row_idx = next(
                (i for i in range(0, 5) if sheet_df.iloc[i, 1:8].notna().all()),
                1,  # fallback
            )
            years = sheet_df.iloc[year_row_idx, 1:8].values

            # Find where weeks start by checking column A
            week_start_row = next(
                (
                    i
                    for i in range(year_row_idx + 1, sheet_df.shape[0])
                    if isinstance(sheet_df.iloc[i, 0], str)
                    or pd.notna(sheet_df.iloc[i, 0])
                ),
                year_row_idx + 1,
            )

            for col_idx, year in enumerate(years, start=1):
                for row in range(week_start_row, sheet_df.shape[0]):
                    week = sheet_df.iloc[row, 0]
                    value = sheet_df.iloc[row, col_idx]

                    # Clean week
                    if isinstance(week, str):
                        week = week.strip()
                        if week.isdigit():
                            week_clean = int(week)
                        elif week.lower().startswith(("w", "s")):
                            digits = re.sub(r"\D", "", week)
                            week_clean = int(digits) if digits else None
                        else:
                            week_clean = None
                    elif pd.notna(week):
                        week_clean = int(week)
                    else:
                        week_clean = None

                    cleaned_data.append(
                        {
                            "province": "Nampula",
                            "district": sheet.strip(),
                            "year": int(year),
                            "week": week_clean,
                            "cases": value,
                        }
                    )

# Combine into DataFrame
cholera_df = pd.DataFrame(cleaned_data)
cholera_df = cholera_df.sort_values(["province", "district", "year", "week"])

In [60]:
cholera_df.to_csv(
    Path(os.getenv("AA_DATA_DIR"))
    / "private"
    / "processed"
    / "moz"
    / "cholera"
    / "cholera_data_cleaned.csv",
    index=False,
)

In [ ]:
# csv_buffer = io.StringIO()
# cholera_df.to_csv(csv_buffer, index=False)
# csv_bytes = csv_buffer.getvalue().encode("utf-8")

# file_name = "cholera_data_cleaned.csv"
# blob_write_url = (
#    DEV_BLOB_URL
#    + CONTAINER_NAME
#    + "/"
#    + BLOB_PATH_WRITE
#    + file_name
#    + "?"
#    + DEV_BLOB_SAS
# )
# blob_client = BlobClient.from_blob_url(blob_url)
# blob_client.upload_blob(csv_bytes, overwrite=True)